<a href="https://colab.research.google.com/github/Djadih/ECE381V-Genomic-SP-DS-HW5/blob/main/hw5_p1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://www.ece.utexas.edu/~hvikalo/ee381v/SNP_Fragment_Matrix.txt
!wget http://www.ece.utexas.edu/~hvikalo/ee381v/YeastCycle.xls

--2022-04-20 20:53:47--  http://www.ece.utexas.edu/~hvikalo/ee381v/SNP_Fragment_Matrix.txt
Resolving www.ece.utexas.edu (www.ece.utexas.edu)... 128.83.21.174
Connecting to www.ece.utexas.edu (www.ece.utexas.edu)|128.83.21.174|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://users.ece.utexas.edu/~hvikalo/ee381v/SNP_Fragment_Matrix.txt [following]
--2022-04-20 20:53:47--  http://users.ece.utexas.edu/~hvikalo/ee381v/SNP_Fragment_Matrix.txt
Resolving users.ece.utexas.edu (users.ece.utexas.edu)... 146.6.225.99
Connecting to users.ece.utexas.edu (users.ece.utexas.edu)|146.6.225.99|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15600 (15K)
Saving to: ‘SNP_Fragment_Matrix.txt.1’

SNP_Fragment_Matrix 100%[===================>]  15.23K  --.-KB/s    in 0.04s   

2022-04-20 20:53:47 (420 KB/s) - ‘SNP_Fragment_Matrix.txt.1’ saved [15600/15600]

--2022-04-20 20:53:47--  http://www.ece.utexas.edu/~hvikalo/ee381v/YeastCycle.xls


In [ ]:
import numpy as np
np.set_printoptions(linewidth=np.inf)

In [ ]:
f = open("/content/SNP_Fragment_Matrix.txt",'r')

hap_1 = {}
hap_2 = {}
R = np.zeros((80,194))
for line_num,line in enumerate(f):
  for idx, chr in enumerate(line.strip()):
    if chr != "-":
      if idx not in hap_1:
        hap_1[idx] = chr
        R[line_num,idx] = 1
      if idx in hap_1:
        if hap_1[idx] == chr:
          R[line_num,idx] = 1
        else:
          R[line_num,idx] = -1
          hap_2[idx] = chr

In [ ]:
hap_2

In [ ]:
mask = R!=0

In [ ]:
R

In [ ]:
mask = mask.astype(np.int32)
mask

In [ ]:
(mask * R) == R

In [ ]:
np.set_printoptions(linewidth=np.inf)
for idx, vec in enumerate(R):
  print(idx, np.count_nonzero(vec==-1))


In [ ]:
def f(mask,R,U,V):
  return np.linalg.norm(mask * (R - U @ V.T))

def argmin_F_U(mask, R, U, V):
  for row in range(U.shape[0]):
    U_copy = U.copy()
    U_copy[row] = U_copy[row] ^ 1
    f_orig = f(mask,R,U,V)
    f_mod = f(mask,R,U_copy,V)
    if f_mod < f_orig:
      U = U_copy
  return U


In [ ]:
alpha = 0.02
n = 80
m = 194
k = 2
#init U and V
U = np.random.randint(0,2,size = (n,1))
U = np.concatenate([U,np.bitwise_xor(U,1)],axis=1)

V = (np.random.randint(0,2,size = (m,1))*2-1).astype(np.int32)
V = np.concatenate([V,-V],axis=1)

for iteration in range(500):
  d_f = -2 * (mask * (R - U @ V.T)).T @ U
  V = V - alpha * d_f
  U = argmin_F_U(mask, R, U, V)


In [ ]:
U

In [ ]:
V

In [ ]:
V_quant = np.rint(V)

In [ ]:
V_quant[0,1] = 1
V_quant

In [ ]:
h1 = V_quant.T[0]
h2 = V_quant.T[1]

In [ ]:
s = ''
for i, j in enumerate(h2):
  if(j==1):
    s+=hap_1[i]
  if(j==-1):
    s+=hap_2[i]

In [ ]:
s = ''
for i in range(194):
  s += h1[i]

In [ ]:
s

'AGACTATCGGTATAAAGCCTCGTGAGATCCATCCGGACATCCGATGAGAAGGAGTTCAGCCTACCCTACACGAACGGCTCTTGCCACGCCTGGAGGTACTTTCGCGCCCCCAAATTACGATTGTCTTTGGAGACAGCCGGTTTCGCAGCTGCGAAAACGATCCGGTCGATGTCCGTAGGAAGGAAAAGCCCGAA'

In [ ]:
def hd(row, h):
  d = 0
  for idx,i in enumerate(row):
    if(i != 0):
      if(i!=h[idx]):
        d+=1
  return d

def MEC(R,h1,h2):
  d = 0
  for row in R:
    d+=min(hd(row,h1),hd(row,h2))
  return d

In [ ]:
MEC(R,h1,h2)

195